# EEGGAN Test

EEGGAN is a good (untested) EEG GAN library that still needs to be tested even after writing this notebook.

I recommend instead of looking at this notebook, it'd be better to consult the EEGGAN Documentation written [here](https://autoresearch.github.io/EEG-GAN/Tutorials/Tutorial_AEGAN/).

In [ ]:
import os
import mne
import numpy as np
import glob
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# load single file

# load processed fif file
processed_dir = os.path.join('..', '..', 'filtered_processed')
resting_files = glob.glob(os.path.join(processed_dir, "*resting_state*.fif"))

raw = resting_files[0]

raw = mne.io.read_raw_fif(raw, preload=True)

In [ ]:
# Extract events for each raw file separately
task_1_events_list = []
task_2_events_list = []

events, event_dict = mne.events_from_annotations(raw)
    
# Extract eyes closed events (event_id 3)
task_1 = events[events[:, 2] == 3]
task_1_events_list.append(task_1)
    
# Extract eyes open events (event_id 4)
task_2 = events[events[:, 2] == 4]
task_2_events_list.append(task_2)

In [ ]:
# Extract signal data for each event
# Get the first event sample for each task
task1_sample = task_1[0, 0]
task2_sample = task_2[0, 0]

# Define epoch duration (e.g., 1 second = 500 samples at 500Hz)
duration_samples = 1000  # adjust based on your sampling rate
task1_data = raw.get_data(start=task1_sample, stop=task1_sample + duration_samples)
task2_data = raw.get_data(start=task2_sample, stop=task2_sample + duration_samples)

# Average across channels
task1_waveform = task1_data.mean(axis=0)
task2_waveform = task2_data.mean(axis=0)

# Create time array matching the data length
time = np.linspace(0, duration_samples / raw.info['sfreq'] * 1000, duration_samples)

#Setup figure
f, (ax1) = plt.subplots(1, 1, figsize=(6, 4))

#Plot each waveform
ax1.plot(time, task1_waveform, label = 'Task 1 (Eyes Closed)')
ax1.plot(time, task2_waveform, label = 'Task 2 (Eyes Open)')

#Format plot
ax1.set_ylabel('Voltage ($\mu$V)')
ax1.set_xlabel('Time (ms)')
ax1.set_title('Empirical', loc='left')
ax1.spines[['right', 'top']].set_visible(False)
ax1.legend(frameon=False)
plt.tight_layout()
plt.show()

In [ ]:
# prepare data
sfreq = raw.info['sfreq']
epoch_duration = 2.0  # seconds
samples_per_epoch = int(epoch_duration * sfreq)

print(f"Sampling rate: {sfreq} Hz")
print(f"Epoch duration: {epoch_duration}s = {samples_per_epoch} samples")

# Extract multiple trials per condition with overlap
n_trials_per_condition = 50
trial_overlap = 0.5
step_size = int(samples_per_epoch * (1 - trial_overlap))

all_data = []

for trial_idx in range(n_trials_per_condition):
    start_sample = task_1[0, 0] + (trial_idx * step_size)
    end_sample = start_sample + samples_per_epoch
    
    if end_sample > raw.n_times:
        break
        
    trial_data = raw.get_data(start=start_sample, stop=end_sample)
    
    for ch_idx, ch_name in enumerate(raw.ch_names):
        row = {'Condition': 0, 'Electrode': ch_idx + 1}
        for t_idx in range(samples_per_epoch):
            row[f'Time{t_idx + 1}'] = float(trial_data[ch_idx, t_idx])
        all_data.append(row)

for trial_idx in range(n_trials_per_condition):
    start_sample = task_2[0, 0] + (trial_idx * step_size)
    end_sample = start_sample + samples_per_epoch
    
    if end_sample > raw.n_times:
        break
        
    trial_data = raw.get_data(start=start_sample, stop=end_sample)
    
    for ch_idx, ch_name in enumerate(raw.ch_names):
        row = {'Condition': 1, 'Electrode': ch_idx + 1}
        for t_idx in range(samples_per_epoch):
            row[f'Time{t_idx + 1}'] = float(trial_data[ch_idx, t_idx])
        all_data.append(row)

df = pd.DataFrame(all_data)

df['Condition'] = df['Condition'].astype(np.int32)
df['Electrode'] = df['Electrode'].astype(np.int32)

time_cols = [col for col in df.columns if col.startswith('Time')]
for col in time_cols:
    df[col] = df[col].astype(np.float64)

print(f"\n=== Data Summary ===")
print(f"DataFrame shape: {df.shape}")
print(f"Samples per condition: {len(df[df['Condition'] == 0]) // len(raw.ch_names)}")
print(f"Channels: {len(raw.ch_names)}")
print(f"Timepoints per trial: {samples_per_epoch}")

data_path = 'resting_state_eeggan_format.csv'
df.to_csv(data_path, index=False)

df_test = pd.read_csv(data_path)
print(f"\n=== Verification ===")
print(f"Reloaded shape: {df_test.shape}")
print(f"Dtypes correct: Condition={df_test['Condition'].dtype}, Time1={df_test['Time1'].dtype}")
print(f"✓ Data saved: {data_path}")

emp_time_start = 2
time_ms = np.linspace(0, epoch_duration * 1000, samples_per_epoch)

In [ ]:
# DIAGNOSTIC: Verify condition differences exist in training data
from sklearn.preprocessing import scale

time_cols = [col for col in df.columns if col.startswith('Time')]

# Check if conditions are actually different in the data
closed_mean = df[df['Condition'] == 0][time_cols].mean(axis=0).values
open_mean = df[df['Condition'] == 1][time_cols].mean(axis=0).values

mean_diff = np.abs(closed_mean - open_mean).mean()
closed_std = df[df['Condition'] == 0][time_cols].values.std()
open_std = df[df['Condition'] == 1][time_cols].values.std()

print("=" * 70)
print("TRAINING DATA DIAGNOSTICS")
print("=" * 70)
print(f"Eyes Closed samples: {len(df[df['Condition'] == 0])}")
print(f"Eyes Open samples: {len(df[df['Condition'] == 1])}")
print(f"\nCondition Statistics:")
print(f"  Eyes Closed: Mean={closed_mean.mean():.6f}, Std={closed_std:.6f}")
print(f"  Eyes Open:   Mean={open_mean.mean():.6f}, Std={open_std:.6f}")
print(f"  Absolute difference: {mean_diff:.6f}")
print(f"  Effect size (Cohen's d): {(closed_mean.mean() - open_mean.mean()) / ((closed_std + open_std) / 2):.3f}")

if mean_diff < 1e-6:
    print("\n⚠️ WARNING: Conditions are TOO SIMILAR!")
    print("   The GAN will not learn meaningful differences.")
    print("   Possible solutions:")
    print("   1. Extract epochs during actual task performance (not just at event markers)")
    print("   2. Use longer epochs to capture more temporal dynamics")
    print("   3. Include more channels to capture spatial differences")
else:
    print(f"\n✓ Conditions show {mean_diff:.6f} µV difference - should be trainable!")

# Visualize the difference
plt.figure(figsize=(12, 5))
ax1 = plt.subplot(1, 2, 1)
ax1.plot(time_ms, closed_mean, label='Eyes Closed', linewidth=2, color='C0')
ax1.plot(time_ms, open_mean, label='Eyes Open', linewidth=2, color='C1')
ax1.set_xlabel('Time (ms)', fontsize=11)
ax1.set_ylabel('Amplitude (µV)', fontsize=11)
ax1.set_title('Raw Condition Averages', fontweight='bold')
ax1.legend(frameon=False)
ax1.grid(alpha=0.3)
ax1.spines[['right', 'top']].set_visible(False)

ax2 = plt.subplot(1, 2, 2)
ax2.plot(time_ms, scale(closed_mean), label='Eyes Closed', linewidth=2, color='C0')
ax2.plot(time_ms, scale(open_mean), label='Eyes Open', linewidth=2, color='C1')
ax2.plot(time_ms, scale(closed_mean - open_mean), label='Difference', 
         linewidth=2, linestyle='--', alpha=0.7, color='C2')
ax2.set_xlabel('Time (ms)', fontsize=11)
ax2.set_ylabel('Standardized Amplitude', fontsize=11)
ax2.set_title('Standardized Comparison', fontweight='bold')
ax2.legend(frameon=False)
ax2.grid(alpha=0.3)
ax2.spines[['right', 'top']].set_visible(False)

plt.tight_layout()
plt.savefig('condition_diagnostics.png', dpi=300, bbox_inches='tight')
plt.show()
print("\n✓ Saved: condition_diagnostics.png")

In [ ]:
# Train autoencoder
save_name = "autoencoder_resting_state.pt"
kw_channel = 'Electrode'
target = 'time'
time_out = 50
n_epochs = 10
seed = 42

!eeggan autoencoder_training data={data_path} save_name={save_name} kw_channel={kw_channel} kw_time={kw_time} target={target} time_out={time_out} n_epochs={n_epochs} seed={seed}

In [ ]:
# Train GAN - using tutorial-recommended settings
save_name = 'gan_resting_state.pt'
autoencoder = 'trained_ae/autoencoder_resting_state.pt'
kw_conditions = 'Condition'
kw_channel = 'Electrode'
patch_size = 10 
n_epochs = 10
seed = 42

!eeggan gan_training data={data_path} save_name={save_name} autoencoder={autoencoder} kw_conditions={kw_conditions} kw_channel={kw_channel} patch_size={patch_size} n_epochs={n_epochs} seed={seed}

In [ ]:
# Visualize training results
print("=== Visualizing Autoencoder ===")
!eeggan visualize average model=trained_ae/autoencoder_resting_state.pt

print("\n=== Visualizing GAN Training Losses ===")
!eeggan visualize loss model=trained_models/gan_resting_state.pt

print("\n=== Visualizing GAN Generated Waveforms ===")
!eeggan visualize average model=trained_models/gan_resting_state.pt

In [ ]:
# Generate synthetic samples
num_samples = 1000

print(f"Generating {num_samples} samples per condition...\n")

# Eyes Closed (Condition 0)
!eeggan generate_samples model=trained_models/gan_resting_state.pt conditions=0 save_name=synthetic_eyes_closed.csv num_samples_total={num_samples} seed=42

# Eyes Open (Condition 1)
!eeggan generate_samples model=trained_models/gan_resting_state.pt conditions=1 save_name=synthetic_eyes_open.csv num_samples_total={num_samples} seed=42

print("\n✓ Synthetic samples generated!")
print("Files saved in generated_samples/:")
print("  - synthetic_eyes_closed.csv")
print("  - synthetic_eyes_open.csv")

In [ ]:
# Load and filter synthetic data
from scipy import signal
from sklearn.preprocessing import scale
import random as rnd

rnd.seed(42)

def bandpass_filter_eeg(data, fs, lowcut=0.5, highcut=40):
    """Apply bandpass filter to EEG data"""
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(4, [low, high], btype='band')
    return signal.filtfilt(b, a, data)

# Load synthetic data
print("Loading synthetic data...")
syn_closed = np.genfromtxt('generated_samples/synthetic_eyes_closed.csv', delimiter=',', skip_header=1)
syn_open = np.genfromtxt('generated_samples/synthetic_eyes_open.csv', delimiter=',', skip_header=1)
syntheticEEG = np.vstack([syn_closed, syn_open])

print(f"Synthetic shape: {syntheticEEG.shape}")

# Filter time columns (skip Condition and Electrode)
syn_time_start = 2
print("Applying bandpass filter...")
for i in range(syntheticEEG.shape[0]):
    syntheticEEG[i, syn_time_start:] = bandpass_filter_eeg(
        syntheticEEG[i, syn_time_start:], 
        fs=sfreq
    )

print("✓ Filtering complete!")

# Compare individual trials
empiricalIndex = rnd.sample(range(len(df)), 5)
syntheticIndex = rnd.sample(range(len(syntheticEEG)), 5)

f, axes = plt.subplots(5, 2, figsize=(14, 10))

for c in range(5):
    # Empirical
    emp_data = df.iloc[empiricalIndex[c], emp_time_start:].values
    axes[c, 0].plot(time_ms, emp_data, linewidth=1.5, color='C0')
    axes[c, 0].set_yticks([])
    
    # Synthetic (filtered)
    syn_data = syntheticEEG[syntheticIndex[c], syn_time_start:]
    axes[c, 1].plot(time_ms, syn_data, linewidth=1.5, color='C1')
    axes[c, 1].set_yticks([])
    axes[c, 1].spines['left'].set_visible(False)
    
    if c == 0:
        axes[c, 0].set_title('Empirical', loc='left', fontweight='bold', fontsize=12)
        axes[c, 1].set_title('Synthetic (Filtered)', loc='left', fontweight='bold', fontsize=12)
    
    if c != 4:
        for ax in axes[c]:
            ax.spines[['bottom', 'left', 'right', 'top']].set_visible(False)
            ax.set_xticks([])
    else:
        for ax in axes[c]:
            ax.spines[['left', 'right', 'top']].set_visible(False)
            ax.set_xlabel('Time (ms)', fontsize=11)

plt.tight_layout()
plt.savefig('trial_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved: trial_comparison.png")

In [ ]:
# Grand average comparison
time_cols = [col for col in df.columns if col.startswith('Time')]

# Compute grand averages
emp_closed = df[df['Condition'] == 0][time_cols].mean(axis=0).values
emp_open = df[df['Condition'] == 1][time_cols].mean(axis=0).values

syn_closed = syntheticEEG[syntheticEEG[:, 0] == 0, syn_time_start:].mean(axis=0)
syn_open = syntheticEEG[syntheticEEG[:, 0] == 1, syn_time_start:].mean(axis=0)

# Plot comparison
f, axes = plt.subplots(1, 2, figsize=(14, 5))

# Empirical
axes[0].plot(time_ms, scale(emp_closed), label='Eyes Closed', linewidth=2.5, color='C0')
axes[0].plot(time_ms, scale(emp_open), label='Eyes Open', linewidth=2.5, color='C1')
axes[0].set_ylabel('Standardized Amplitude', fontsize=12)
axes[0].set_xlabel('Time (ms)', fontsize=12)
axes[0].set_title('Empirical (Original)', loc='left', fontweight='bold', fontsize=13)
axes[0].spines[['right', 'top']].set_visible(False)
axes[0].legend(frameon=False, fontsize=11)
axes[0].grid(alpha=0.3)

# Synthetic
axes[1].plot(time_ms, scale(syn_closed), label='Eyes Closed', linewidth=2.5, color='C0')
axes[1].plot(time_ms, scale(syn_open), label='Eyes Open', linewidth=2.5, color='C1')
axes[1].set_ylabel('Standardized Amplitude', fontsize=12)
axes[1].set_xlabel('Time (ms)', fontsize=12)
axes[1].set_title('Synthetic (GAN Generated)', loc='left', fontweight='bold', fontsize=13)
axes[1].spines[['right', 'top']].set_visible(False)
axes[1].legend(frameon=False, fontsize=11)
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig('grand_average_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved: grand_average_comparison.png")

# Statistics
from scipy.stats import pearsonr

print("\n" + "="*60)
print("GRAND AVERAGE STATISTICS")
print("="*60)
print("\nEmpirical:")
print(f"  Eyes Closed: Mean={emp_closed.mean():.4f}, Std={emp_closed.std():.4f}")
print(f"  Eyes Open:   Mean={emp_open.mean():.4f}, Std={emp_open.std():.4f}")
# ADD: Show difference between conditions
emp_diff = np.abs(emp_closed - emp_open).mean()
print(f"  Mean absolute difference: {emp_diff:.4f}")

print("\nSynthetic (Filtered):")
print(f"  Eyes Closed: Mean={syn_closed.mean():.4f}, Std={syn_closed.std():.4f}")
print(f"  Eyes Open:   Mean={syn_open.mean():.4f}, Std={syn_open.std():.4f}")
# ADD: Show difference between conditions
syn_diff = np.abs(syn_closed - syn_open).mean()
print(f"  Mean absolute difference: {syn_diff:.4f}")

# ADD: Ratio of synthetic to empirical difference
if emp_diff > 0:
    diff_ratio = syn_diff / emp_diff
    print(f"\n⚠️ Condition separation: {diff_ratio*100:.1f}% of empirical")
    if diff_ratio < 0.3:
        print("   → GAN is NOT learning condition differences well")
        print("   → Try training for MORE epochs (2000+)")
        print("   → Or increase autoencoder compression to preserve differences")

corr_closed, p_closed = pearsonr(scale(emp_closed), scale(syn_closed))
corr_open, p_open = pearsonr(scale(emp_open), scale(syn_open))
print("\nCorrelation (Empirical vs Synthetic):")
print(f"  Eyes Closed: r={corr_closed:.3f} (p={p_closed:.4f})")
print(f"  Eyes Open:   r={corr_open:.3f} (p={p_open:.4f})")
print("="*60)

In [ ]:
# Power Spectral Density comparison
from scipy import signal as sig

def compute_psd(data, fs):
    """Compute average PSD across all samples"""
    psds = []
    for i in range(data.shape[0]):
        f, psd = sig.welch(data[i], fs=fs, nperseg=min(256, len(data[i])))
        psds.append(psd)
    return f, np.mean(psds, axis=0)

print("Computing Power Spectral Density...")
f_emp, psd_emp = compute_psd(df[time_cols].values, sfreq)
f_syn, psd_syn = compute_psd(syntheticEEG[:, syn_time_start:], sfreq)

plt.figure(figsize=(10, 5))
plt.semilogy(f_emp, psd_emp, label='Empirical', linewidth=2, color='C0')
plt.semilogy(f_syn, psd_syn, label='Synthetic (Filtered)', linewidth=2, color='C1')
plt.xlabel('Frequency (Hz)', fontsize=11)
plt.ylabel('Power Spectral Density', fontsize=11)
plt.title('Power Spectral Density Comparison', fontweight='bold', fontsize=12)
plt.legend(frameon=False, fontsize=11)
plt.xlim([0, 50])
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('psd_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("✓ Saved: psd_comparison.png")

print("\n" + "="*60)
print("ANALYSIS COMPLETE!")
print("="*60)
print("\nGenerated files:")
print("  1. trial_comparison.png")
print("  2. grand_average_comparison.png")
print("  3. psd_comparison.png")
print("\nYou can now use the synthetic data for augmentation!")